In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
import sys
import time
import sklearn
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for model in mpl, np, pd, sklearn, keras:
    print(model.__name__, model.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)
matplotlib 3.2.2
numpy 1.19.0
pandas 1.0.5
sklearn 0.23.1
tensorflow_core.keras 2.2.4-tf


In [2]:
# tf_function and tuto-graph
def scaled_elu(z, scale=1.0, alpha=1.0):
    # z >=0 ? scale * z : scale * alpha * tf.nn.elu(z)
    is_positive = tf.greater_equal(z,0.0)
    return scale * tf.where(is_positive, z , alpha * tf.nn.elu(z))

print(scaled_elu(tf.constant(-3.)))
print(scaled_elu(tf.constant([-1.0,3])))
    
scaled_elu_tf = tf.function(scaled_elu) # tf.function --> py function --> tf graph  tensorflow图结构-函数
print(scaled_elu_tf(tf.constant(-3.0)))
print(scaled_elu_tf(tf.constant([-1.0,3])))

print(scaled_elu_tf.python_function is scaled_elu) # to XXX.python_function

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.63212055  3.        ], shape=(2,), dtype=float32)
tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.63212055  3.        ], shape=(2,), dtype=float32)
True


In [3]:
%timeit scaled_elu(tf.random.normal((1000,1000)))
%timeit scaled_elu_tf(tf.random.normal((1000,1000))) # 快 tf.function

33.5 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
31.3 ms ± 3.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
# 1 + 1/2 + 1/2^2 + .....+ 1/2^n
# tf.function或者 @tf.function --> py function --> tf graph  tensorflow图结构-函数
@ tf.function
def converge_to_2(n_iters):
    total = tf.constant(0.) # tf.constant()函数内部可行
    increment = tf.constant(1.)
    for i in range(n_iters):
        total += increment
        increment /= 2.0
    return total

print(converge_to_2(20))



tf.Tensor(1.9999981, shape=(), dtype=float32)


In [5]:
# tf.autograph.to_code()
# tf.autograph.to_graph()
from IPython.display import display, Markdown
def display_tf_code(func):
    code = tf.autograph.to_code(func)
    display(Markdown('```python\n{}\n```'.format(code)))


In [6]:
display_tf_code(scaled_elu)

```python
def tf__scaled_elu(z, scale=None, alpha=None):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('scaled_elu', 'scaled_elu_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as scaled_elu_scope:
    is_positive = ag__.converted_call(tf.greater_equal, scaled_elu_scope.callopts, (z, 0.0), None, scaled_elu_scope)
    do_return = True
    retval_ = scaled_elu_scope.mark_return_value(scale * ag__.converted_call(tf.where, scaled_elu_scope.callopts, (is_positive, z, alpha * ag__.converted_call(tf.nn.elu, scaled_elu_scope.callopts, (z,), None, scaled_elu_scope)), None, scaled_elu_scope))
  do_return,
  return ag__.retval(retval_)

```

In [7]:
display(scaled_elu_tf)

In [9]:
display(converge_to_2)

In [10]:
def display_tf_graph(func):
    code = tf.autograph.to_graph(func)
    display(Markdown('```python\n{}\n```'.format(code)))


In [11]:
display_tf_graph(scaled_elu)

```python
<function create_converted_entity_factory.<locals>.create_converted_entity.<locals>.tf__scaled_elu at 0x00000000214D2678>
```

In [12]:
# tf.Variable 不能在 @tf.function()的函数内部
var = tf.Variable(0.0) 
@tf.function
def add_21():
    # var = tf.Variable(0.0) # 当tf.function时，不能在函数内部 定义变量
    return var.assign_add(21)

print(add_21())

tf.Tensor(21.0, shape=(), dtype=float32)


In [13]:
# 立方
@tf.function(input_signature =[tf.TensorSpec([None], tf.int32, name='x')])
def cube(z):
    return tf.pow(z,3)

try:
    print(cube(tf.constant([1.,2.,3.])))
except ValueError as ex:
    print(ex)
    
print(cube(tf.constant([1,2,3])))


Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='x'))
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [14]:
# @tf.function  py func--> tf graph
# get_concrete_function --> add input signature --> SaveModel

cube_func_int32 = cube.get_concrete_function(tf.TensorSpec([None],tf.int32))
print(cube_func_int32)

In [16]:
print(cube_func_int32 is cube.get_concrete_function(tf.TensorSpec([5],tf.int32)))

print(cube_func_int32 is cube.get_concrete_function(tf.constant([1,2,3])))

True
True


In [17]:
cube_func_int32.graph

In [18]:
cube_func_int32.graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [19]:
pow_op = cube_func_int32.graph.get_operations()[2]
print(pow_op)

name: "Pow"
op: "Pow"
input: "x"
input: "Pow/y"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



In [20]:
print(list(pow_op.inputs))
print(list(pow_op.outputs))

[<tf.Tensor 'x:0' shape=(None,) dtype=int32>, <tf.Tensor 'Pow/y:0' shape=() dtype=int32>]
[<tf.Tensor 'Pow:0' shape=(None,) dtype=int32>]


In [22]:
cube_func_int32.graph.get_operation_by_name('x')

<tf.Operation 'x' type=Placeholder>

In [23]:
cube_func_int32.graph.get_tensor_by_name("x:0")

<tf.Tensor 'x:0' shape=(None,) dtype=int32>

In [24]:
cube_func_int32.graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "x"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 119
}